In [1]:
import warnings

warnings.filterwarnings('ignore')

In [2]:
import pickle
import random as rnd
import warnings
from math import exp, log

import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sn
import seaborn as sns
import sklearn
import statsmodels.api as sm
import tensorflow as tf
import xgboost as xgb
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from keras.layers import *
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasRegressor
from lightgbm import LGBMClassifier
from matplotlib import pyplot
from matplotlib.font_manager import FontProperties
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import (LinearDiscriminantAnalysis,
                                           QuadraticDiscriminantAnalysis)
from sklearn.ensemble import (AdaBoostClassifier, BaggingClassifier,
                              ExtraTreesClassifier, GradientBoostingClassifier,
                              RandomForestClassifier, VotingClassifier)
from sklearn.feature_selection import (RFE, SelectFromModel, SelectKBest,
                                       VarianceThreshold, chi2)
from sklearn.inspection import permutation_importance
from sklearn.linear_model import (LinearRegression, LogisticRegression,
                                  PassiveAggressiveClassifier, Perceptron,
                                  RidgeClassifier, SGDClassifier)
from sklearn.metrics import (accuracy_score, balanced_accuracy_score,
                             classification_report, confusion_matrix, log_loss,
                             precision_score, recall_score, roc_auc_score)
from sklearn.model_selection import (GridSearchCV, KFold, RandomizedSearchCV,
                                     ShuffleSplit, StratifiedKFold,
                                     cross_val_score, train_test_split)
from sklearn.naive_bayes import (BernoulliNB, ComplementNB, GaussianNB,
                                 MultinomialNB)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (Binarizer, LabelEncoder, MinMaxScaler,
                                   Normalizer, OneHotEncoder,
                                   PolynomialFeatures, StandardScaler,
                                   normalize)
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from xgboost import XGBClassifier, XGBRFClassifier

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [7]:
trainADR = pd.read_csv("train_adr_nn.csv")
traindf = pd.read_csv("train_for_predict_adr.csv")
trainlabel = pd.read_csv("train_label.csv")

In [8]:
traindf["adr"] = trainADR.values
traindf["revenue"] = traindf["adr"] * traindf["total_nights"]
traindf.head()

,hotel,is_canceled,lead_time,arrival_date_week_number,stays_in_weekend_nights,stays_in_week_nights,adults,children,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests,date,arrival_date_month_April,arrival_date_month_August,arrival_date_month_December,arrival_date_month_February,arrival_date_month_January,arrival_date_month_July,arrival_date_month_June,arrival_date_month_March,arrival_date_month_May,arrival_date_month_November,arrival_date_month_October,arrival_date_month_September,is_weekend,is_same_room,reserved_A,reserved_B,reserved_C,reserved_D,reserved_E,reserved_F,reserved_G,reserved_H,reserved_L,assigned_A,assigned_B,assigned_C,assigned_D,assigned_E,assigned_F,assigned_G,assigned_H,assigned_I,assigned_K,total_nights,long_stay,total_customers,lowAdrCountry,midAdrCountry,highAdrCountry,total_previos_booking,meal_BB,meal_FB,meal_HB,meal_SC,meal_Undefined,market_segment_Aviation,market_segment_Complementary,market_segment_Corporate,market_segment_Direct,market_segment_Groups,market_segment_Offline TA/TO,market_segment_Online TA,distribution_channel_Corporate,distribution_channel_Direct,distribution_channel_GDS,distribution_channel_TA/TO,distribution_channel_Undefined,deposit_type_No Deposit,deposit_type_Non Refund,deposit_type_Refundable,customer_type_Contract,customer_type_Group,customer_type_Transient,customer_type_Transient-Party,revenue
0,1,0,257,27,0,2,1,0.00,0,0,0,1,0,56.76,0,0,2015-07-01,0,0,0,0,0,1,0,0,0,0,0,0,0.00,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0.00,1.00,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,113.53
1,1,0,257,27,0,2,2,0.00,0,0,0,0,0,80.19,0,0,2015-07-01,0,0,0,0,0,1,0,0,0,0,0,0,0.00,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0.00,2.00,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,160.39
2,1,0,257,27,0,2,2,0.00,0,0,0,0,0,80.19,0,0,2015-07-01,0,0,0,0,0,1,0,0,0,0,0,0,0.00,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0.00,2.00,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,160.39
3,1,0,257,27,0,2,2,0.00,0,0,0,0,0,80.19,0,0,2015-07-01,0,0,0,0,0,1,0,0,0,0,0,0,0.00,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0.00,2.00,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,160.39
4,1,0,257,27,0,2,2,0.00,0,0,0,0,0,80.19,0,0,2015-07-01,0,0,0,0,0,1,0,0,0,0,0,0,0.00,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0.00,2.00,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,160.39


In [9]:
df = trainlabel.copy()
df["revenue"] = traindf.groupby("date")["revenue"].agg("sum").values
df.groupby("label")["revenue"].agg(["mean", "count", "max", "min", "std"])

,mean,count,max,min,std
label,,,,,
0.00,7444.93,94,13006.35,2213.85,2242.88
1.00,16171.53,152,24686.90,10667.59,3137.07
2.00,25129.85,186,36298.20,18861.52,3414.31
3.00,34999.01,124,48205.79,28945.37,3469.58
4.00,44471.60,46,52844.35,34493.96,3862.17
5.00,51766.46,21,60315.45,36349.01,5708.01
6.00,61827.68,13,69181.40,47116.98,5969.49
7.00,75083.57,1,75083.57,75083.57,nan
8.00,80397.81,2,82728.14,78067.48,3295.58


In [10]:
df.head()

,arrival_date,label,revenue
0,2015-07-01,2.00,27258.44
1,2015-07-02,1.00,19508.48
2,2015-07-03,1.00,15680.18
3,2015-07-04,1.00,23081.99
4,2015-07-05,1.00,24686.90


In [11]:
df["label"] = df["label"].replace({7: 6, 8: 6, 9: 6})

In [12]:
new_train_x = df.drop(["arrival_date", "label"], axis=1)
new_train_y = df["label"]

In [13]:
classifiers = [LogisticRegression(),
               DecisionTreeClassifier(),
               XGBClassifier(),
               ExtraTreesClassifier(),
               RandomForestClassifier(),
               RidgeClassifier(),
               QuadraticDiscriminantAnalysis(),
               ComplementNB(),
               BernoulliNB(),
               GaussianNB(),
               MultinomialNB(),
               LinearDiscriminantAnalysis(),
               SGDClassifier(),
               AdaBoostClassifier(),
               BaggingClassifier(),
               SVC(),
               LinearSVC(),
               KNeighborsClassifier(),
               XGBClassifier(),
               XGBRFClassifier(),
               LGBMClassifier()
               ]

from time import time

for clf in classifiers:
    
    before = time()
    print("="*30)
    print(clf.__class__.__name__)
    
    skf = StratifiedKFold(n_splits=5, shuffle=True)
    crossValScore = cross_val_score(clf, new_train_x, new_train_y, scoring='neg_mean_absolute_error', cv=skf)  
    crossValScore_mean = abs(np.mean(crossValScore)) 
    print("mean_absolute_error: {:.4}".format(crossValScore_mean))
    after = time()
    print(f"花費{(after - before):.1f}s")
    
print("="*30)

LogisticRegression
mean_absolute_error: 1.073
花費0.1s
DecisionTreeClassifier
mean_absolute_error: 0.1891
花費0.0s
XGBClassifier
mean_absolute_error: 0.1953
花費0.4s
ExtraTreesClassifier
mean_absolute_error: 0.2172
花費0.5s
RandomForestClassifier
mean_absolute_error: 0.1984
花費0.6s
RidgeClassifier
mean_absolute_error: 0.5203
花費0.0s
QuadraticDiscriminantAnalysis
mean_absolute_error: 0.1484
花費0.0s
ComplementNB
mean_absolute_error: 2.011
花費0.0s
BernoulliNB
mean_absolute_error: 1.073
花費0.0s
GaussianNB
mean_absolute_error: 0.1484
花費0.0s
MultinomialNB
mean_absolute_error: 1.073
花費0.0s
LinearDiscriminantAnalysis
mean_absolute_error: 0.1516
花費0.0s
SGDClassifier
mean_absolute_error: 1.327
花費0.1s
AdaBoostClassifier
mean_absolute_error: 0.3484
花費0.4s
BaggingClassifier
mean_absolute_error: 0.1969
花費0.1s
SVC
mean_absolute_error: 0.1469
花費0.1s
LinearSVC
mean_absolute_error: 1.445
花費0.3s
KNeighborsClassifier
mean_absolute_error: 0.1703
花費0.0s
XGBClassifier
mean_absolute_error: 0.2016
花費0.4s
XGBRFClassifier
me

In [23]:
model = XGBRFClassifier()
model = model.fit(new_train_x, new_train_y)
y_pred = model.predict(new_train_x)
print(f"Mean absolute error: {metrics.mean_absolute_error(new_train_y, y_pred):.4f}")

Mean absolute error: 0.1094


In [24]:
testdf = pd.read_csv("test_for_predict_adr.csv")
test_label = pd.read_csv("test_nolabel.csv")
test_adr = pd.read_csv("test_adr_nn.csv")

In [25]:
testdf["adr"] = test_adr.values
testdf["revenue"] = testdf["adr"] * testdf["total_nights"]
revenue = testdf.groupby("date")["revenue"].agg("sum")
test_label = test_label.merge(revenue, how="left", left_on="arrival_date", right_on="date")

In [26]:
test_x = test_label.drop(["arrival_date"], axis=1)

In [27]:
test_label["label"] = model.predict(test_x)

In [28]:
test_label[["arrival_date", "label"]].to_csv("answer.csv", index=False)